<a href="https://colab.research.google.com/github/Villainnumber1/ML/blob/main/alvaro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
##############
# Constantes #
##############

#@title  { form-width: "400px" }
IS_TEST = True #@param {type:"boolean"}

---

<h3><center>Cargar librerías</center></h3>

---

In [34]:
import sklearn
import seaborn as sns
import pandas as pd
import statistics as stat
from copy import deepcopy
from random import randint

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, scale, PolynomialFeatures, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

from xgboost import XGBClassifier

!pip install -U imbalanced-learn
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE

# Experimental
!pip install hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

!pip install feature-engine
from feature_engine.selection import (
    DropDuplicateFeatures,
    DropConstantFeatures,
    DropDuplicateFeatures,
    DropCorrelatedFeatures,
    SmartCorrelatedSelection,
    SelectByShuffling,
    SelectBySingleFeaturePerformance,
    RecursiveFeatureElimination,
)


# Configuration
pd.set_option('display.max_columns', 40) # Hacer que pandas muestre 40 columnas de un dataframe

---

<h3><center>Cargar datasets</center></h3>

---

In [35]:
URL_TRAIN_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/train.csv"
URL_TEST_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/test.csv"

IS_TEST = True

X = pd.read_csv(URL_TRAIN_CSV, sep=",")
Y = pd.read_csv(URL_TEST_CSV, sep=",")

X

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,43161,3985,4,Lawrencez,40.0,856431180.0,Accountant,97384.44,8167.370000,1.0,3.0,8.0,1.0,Debt Consolidation Loan,20.0,12.0,0.52,3.0,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909,Standard
1,40663,49494,2,Mohammedr,39.0,417453942.0,Writer,31369.58,2607.131667,8.0,9.0,27.0,8.0,"Home Equity Loan, Not Specified, Not Specified...",30.0,25.0,19.70,6.0,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383,Standard
2,111059,2925,6,Alistairn,17.0,499528888.0,Writer,30171.86,2772.321667,6.0,7.0,30.0,9.0,"Personal Loan, Home Equity Loan, Home Equity L...",49.0,18.0,23.49,10.0,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615,Standard
3,72139,21358,2,Dinesh Naire,22.0,859449371.0,Doctor,13100.02,1246.668333,7.0,5.0,6.0,5.0,"Credit-Builder Loan, Credit-Builder Loan, Not ...",6.0,20.0,10.64,8.0,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832,Standard
4,119841,21221,4,Bakerg,26.0,485239890.0,Doctor,62313.54,5120.795000,6.0,2.0,18.0,4.0,"Auto Loan, Payday Loan, Student Loan, and Cred...",27.0,8.0,12.82,9.0,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,42636,39869,7,Hendrik Sackmannb,50.0,217529489.0,Media_Manager,107761.29,9047.107500,4.0,5.0,9.0,0.0,No Data,7.0,19.0,14.56,3.0,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010,Standard
79996,114031,38305,2,Alexj,33.0,98861902.0,Scientist,16218.32,1098.526667,6.0,6.0,21.0,5.0,"Mortgage Loan, Home Equity Loan, Credit-Builde...",44.0,19.0,8.98,6.0,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798,Poor
79997,85991,9976,6,Luciana Lopezi,25.0,745835485.0,Engineer,29374.77,2377.897500,3.0,7.0,6.0,2.0,"Student Loan, and Payday Loan",23.0,13.0,16.65,7.0,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355,Standard
79998,21240,13193,7,Edward Krudyd,15.0,894419091.0,Manager,43782.45,3543.537500,6.0,9.0,25.0,8.0,"Payday Loan, Auto Loan, Payday Loan, Home Equi...",32.0,16.0,1.88,12.0,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586,Poor


---

<h3><center>Filtrados de datos</center></h3>

---

In [1]:
#############
# Filtrados #
#############

##########################################################################################################################
# Age. Filtrar menores de 15. Probablemente sean datos incorrectos y además Y no tiene ninguna entrada por debajo de 15. #
##########################################################################################################################
#X = X[X["Age"] > 15]

###############################################
# SSN                                         #
# Filtrar a clientes cuyo SSN no es correcto. #
###############################################

def is_valid(ssn):
    return len(ssn) == 9 and ssn[0:3] not in ["000", "333", "666"] and int(ssn[0:3]) <= 772

#X = X[X.apply(lambda row: is_valid(str(int(row["SSN"]))), axis=1)]
#dsadasd

---

<h3><center>Añadir nuevas columnas</center></h3>

---

In [37]:
##############################
# Util functions for columns #
##############################

def add_column(df, columna, f):
  df[columna] = df.apply(lambda row: f(row), axis=1)

def map_column(df, columna, f):
  df[columna] = [ f(val) for val in df[columna] ]

######
# Para meterle todas las categorias de X e Y al OneHotEncoder.
# Si no da problemas de shape porque X e Y tienen diferentes categorias.
######
def unionColumnas(X, Y, columnName):
  return list(set(X[columnName]).union(set(Y[columnName])))

In [38]:
#######
# Age #
#######

##############################################################################
# Definir un rango de edad. TODO: Check si baja significativamente el error. #
##############################################################################
def age_range(row):
    age = row["Age"]
    
    if age < 21:
        return "Minor"
    elif age < 35:
        return "Young Adult"
    elif age < 60:
        return "Adult"
    else:
        return "Old"

for df in X, Y:
    add_column(df, "_Age_Range", age_range)

In [39]:
#######################################
# Separar Type_of_Loan (tested)       #
# x, x, y and z -> [x: 2, y: 1, z: 1] #
#######################################

def countLoans(row, typeOfLoan):
  tol = row["Type_of_Loan"]

  parts = [part[4:] if part.startswith("and") else part for part in tol.split(", ") ]

  return len([True for part in parts if part == typeOfLoan])

#################################################################################
# Frecuencia de loans
#
# Payday Loan: 32372
# Credit-Builder Loan: 32276
# Home Equity Loan: 31238
# Student Loan: 31308
# Mortgage Loan: 31058
# Personal Loan: 31063
# Debt Consolidation Loan: 30922
# Auto Loan: 30391
#
# Frecuencia de Type_of_Loan: Ver drive.
#
# Le he sumado Not Specified y No Data a sus valores más comunes respecivamente
# Not Specified (1 loan) -> Payday Loan
# No Data (0 loans) -> Personal Loan
#################################################################################
for df in X, Y:
  add_column(df, "_Auto Loans",                lambda row: countLoans(row, "Auto Loan"))
  add_column(df, "_Credit-Builder Loans",      lambda row: countLoans(row, "Credit-Builder Loan"))
  add_column(df, "_Debt Consolidation Loans",  lambda row: countLoans(row, "Debt Consolidation Loan"))
  add_column(df, "_Home Equity Loans",         lambda row: countLoans(row, "Home Equity Loan"))
  add_column(df, "_Mortgage Loans",            lambda row: countLoans(row, "Mortgage Loan"))
  add_column(df, "_Payday Loans",              lambda row: countLoans(row, "Payday Loan") + countLoans(row, "Not Specified"))
  
  # Se le suma las no especificadas ya que Personal Loans son el tipo más comun de loans
  add_column(df, "_Personal Loans",            lambda row: countLoans(row, "Personal Loan") + countLoans(row, "No Data"))
  add_column(df, "_Student Loans",             lambda row: countLoans(row, "Student Loan"))
  #add_column(df, "_Not Specified Loans",       lambda row: countLoans(row, "Not Specified"))
  #add_column(df, "_No Data",                   lambda row: countLoans(row, "No Data"))

#X

In [40]:
###############################################
# Razonamiento
#
# Valores mas frecuentes para esta columna:
#   0: 8780x,
#   117659: 8x
#
# Nuff said.
###############################################

for df in X, Y:
    add_column(df, "_Has_EMI_per_month", lambda row: row["Total_EMI_per_month"] != 0)

In [41]:
%%script false --no-raise-error

################################
# EXPERIMENTAL
# FICO:
# 35% payment history
# 30% amount owed
# 15% length of credit history
# 10% new credit
# 10% credit mix
################################
def fico(row, coef):
    cm = { "Bad": 0, "Standard": 1, "Good": 2 }
    
    payment_history = (row["Num_of_Delayed_Payment"] + row["Delay_from_due_date"]) * coef[0] #0.35
    amount_owed = row["Outstanding_Debt"] * coef[1] #0.3
    length_of_credit_history = row["Credit_History_Age"] * coef[2] #0.1
    new_credit = 0 * coef[3]
    credit_mix = cm[row["Credit_Mix"]] * coef[4]# * 100 * 0.1
    
    return payment_history + amount_owed + length_of_credit_history + new_credit + credit_mix

for df in X, Y:
    add_column(df, "_Test", lambda val: fico(val, [0.35, 0.3, 0.1, 0, 0.1]))


In [42]:
%%script false --no-raise-error

####################################################
# Payment Behaviour                                #
#                                                  #
# High_spent_Large_value_payments -> (High, Large) #
# Sube el error                                    #
####################################################
def spent(row):
    behaviour = row["Payment_Behaviour"]
    parts = behaviour.split("_")
    return parts[0]

def value(row):
    behaviour = row["Payment_Behaviour"]
    parts = behaviour.split("_")
    return parts[2]

for df in X, Y:
    add_column(df, "_Payment_Behaviour_Spent", spent)
    add_column(df, "_Payment_Behaviour_Value", value)

In [43]:
###################################################################################################
# Credit_History_Age
#
# Comentario: Es posible que alguien tenga una cuenta abierta por 500 años? Ni que fuera futurama.
###################################################################################################

In [44]:
#################################################################
# Porcentaje de deuda pendiente con respecto al salario mensual #
#################################################################
for df in X, Y:
    add_column(df, "_n_of_months_needed_to_pay_debt", lambda row: row["Outstanding_Debt"]/row["Monthly_Inhand_Salary"])

In [45]:
##########
# Prueba #
##########
areas = [ "New Hampshire", "Maine", "Vermont", "Massachusetts", "Rhode Island", "Connecticut", "New York", "New Jersey", "Pennsylvania", "Maryland", "Delaware",
         "Virginia", "North Carolina", "West Virginia", "Not Issued", "South Carolina", "Georgia", "Florida", "Ohio", "Indiana", "Illinois", "Michigan", "Wisconsin", "Kentucky",
         "Tennessee", "Alabama", "Mississippi", "Arkansas", "Louisiana", "Oklahoma", "Texas", "Minnesota", "Iowa", "Missouri", "North Dakota", "South Dakota",
         "Nebraska", "Kansas", "Montana", "Idaho", "Wyoming", "Colorado", "Arizona", "Utah", "Washington", "Oregon", "California", "Alaska",
         "Hawaii", "District of Columbia", "Virgin Islands", "Puerto Rico", "Guam" ]
areas_cut = [4, 8, 10, 35, 40, 50, 135, 159, 212, 221, 223, 232, 233, 237, 247, 252, 261, 268, 303, 318, 362, 387, 400, 408, 416, 425, 429, 433, 440, 449, 468,
             478, 486, 501, 503, 505, 509, 516, 518, 520, 521, 525, 528, 530, 540, 545, 574, 575, 577, 580, 581, 585, 587]

def area_emision(row):
    ssn = str(row["SSN"])[:3]
    
    if ssn in ["525", "585"]:
        return "New Mexico"
    elif ssn in ["530", "680"]:
        return "Nevada"
    else:
        first_digits = int(ssn)
        
        for i, n in enumerate(areas_cut):
            if first_digits < n:
                return areas[i]
    
        return "Not Issued"

#ssn = X["SSN"]
#for n in ssn:
#    n = str(int(n))
    
#    if area_emision(n) == "Not Issued":
#        print(n, area_emision(n))

for df in X, Y:
    add_column(df, "_SSN_Area_Emision", area_emision)

#
# TODO: Mover arriba
# Adicionalmente, los menores de 12 años no pueden poseer un SSN
X = X[X["_SSN_Area_Emision"] != "Not Issued"]

---

<h3><center>Transformación de columnas</center></h3>

---

In [46]:
# EXPERIMENTAL

#import featuretools as ft

#dataframes = {
#    "clients": (X, "ID")
#}

#relationships = []

#feature_matrix_customers, features_defs = ft.dfs(
#    dataframes=dataframes,
#    relationships=relationships,
#    target_dataframe_name="clients",
#)
#X.shape

In [47]:
#X

In [74]:
normalizer = StandardScaler(with_mean=False)

# TODO: No hay una manera mejor de hacer esto?
keysOccupation  = unionColumnas(X, Y, "Occupation")
keysPMA         = unionColumnas(X, Y, "Payment_of_Min_Amount")
keysNMI         = unionColumnas(X, Y, "Credit_Mix")
keysAIM         = unionColumnas(X, Y, "Payment_Behaviour")
keysTest        = unionColumnas(X, Y, "_SSN_Area_Emision")

# Transformar las columnas
# TODO: Considerar ensembles para mismos clientes?
# TODO: Considerar OrdinalEncoder para features del estilo Good-Normal-Bad
transformersX = [
  ("ID",                                              "drop",                                                       [0]),    # Por razones obvias
  ("Customer_ID",                                     "drop",                                                       [1]),    # Por razones obvias
  ("Month",                                           "drop",                                                       [2]),    # No creo que tenga ninguna influencia
  ("Name",                                            "drop",                                                       [3]),    # No tiene influencia y está repetida
  ("Age",                                             "passthrough",                                                [4]),
  ("SSN",                                             "drop",                                                       [5]),    # No tiene influencia y está repetida
  ("Occupation",                                      OneHotEncoder(categories=[keysOccupation]),                   [6]),    # No deberia influir, pero baja un poco el error? TODO: Comprobar
  ("Annual_Income",                                   "passthrough",                                                [7]),
  ("Monthly_Inhand_Salary",                           "drop",                                                       [8]),    # Annual_Income/12
  ("Num_Bank_Accounts",                               "passthrough",                                                       [9]),    # No tiene influencia
  ("Num_Credit_Card",                                 "passthrough",                                                [10]),
  ("Interest_Rate",                                   "passthrough",                                                [11]),
  ("Num_of_Loan",                                     "passthrough",                                                [12]),
  ("Type_of_Loan",                                    "drop",                                                       [13]),   # Dividida
  ("Delay_from_due_date",                             "passthrough",                                                [14]),
  ("Num_of_Delayed_Payments",                         "passthrough",                                                [15]),
  ("Changed_Credit_Limit",                            "passthrough",                                                [16]),
  ("Num_Credit_Inquiries",                            "passthrough",                                                [17]),
  ("Credit_Mix",                                      OneHotEncoder(categories=[keysNMI]),                          [18]),
  ("Outstanding_Debt",                                "passthrough",                                                [19]),
  ("Credit_Utilization_Ratio",                        "passthrough",                                                [20]),
  ("Credit_History_Age",                              "passthrough",                                                [21]),
  ("Payment_of_Min_Amount",                           OneHotEncoder(categories=[keysPMA]),                          [22]),   # TODO: Considerar binary encoding
  ("Total_EMI_per_Month",                             "passthrough",                                                [23]),
  ("Amount_Invested_Monthly",                         "passthrough",                                                       [24]),   # No tiene influencia
  ("Payment_Behaviour",                               OneHotEncoder(categories=[keysAIM]),                          [25]),
  ("Monthly_Balance",                                 "passthrough",                                                [26]),
  ("Credit_Score",                                    "drop",                                                       [27]),
  
  #####################
  # Columnas añadidas #
  #####################

  # Age
  ("_Age_Range", OneHotEncoder(), [28]),

    
  # Type_of_Loan
  ("_Auto Loans",                                     "passthrough", [29]),
  ("_Credit-Builder Loans",                           "passthrough", [30]),
  ("_Debt Consolidation Loans",                       "passthrough", [31]),
  ("_Home Equity Loans",                              "passthrough", [32]),
  ("_Mortgage Loans",                                 "passthrough", [33]),
  ("_Payday Loans",                                   "passthrough", [34]),
  ("_Personal Loans",                                 "passthrough", [35]),
  ("_Student Loans",                                  "passthrough", [36]),

  # Total_EMI_per_month
  #("_Has_EMI_per_month", "passthrough", [37]),
  #("_Test", "passthrough", [37])
    
  # Payment_Behaviour
  #("_Payment_Spent", "passthrough", [37]),
  #("_Payment_Value", OneHotEncoder(), [38]),
    
  #("_n_of_months_needed_to_pay_debt", "passthrough", [38]),
    
  # SSN
  # This converts X_T into a sparse matrix which needs with_mean=False to parse with standardScaler
  #("_SNS_Area_Emision", OneHotEncoder(categories=[keysTest]), [39])
]

transformersY = deepcopy(transformersX)

# Normbrar el orden de columnas de 0 a n-1
transformersY.pop(27) # Eliminar precio
for i, t in enumerate(transformersY): # Renumerar los indices de las columnas
  transformersY[i][2].pop()
  transformersY[i][2].append(i)

X_T = ColumnTransformer(transformers=transformersX).fit_transform(X)
Y_T = ColumnTransformer(transformers=transformersY).fit_transform(Y)

######################
# Normalize the data #
######################
X_T = normalizer.fit_transform(X_T)
Y_T = normalizer.fit_transform(Y_T)

---

<h3><center>Oversampling</center></h3>

---

In [78]:
_X_T, _Credit_Score = X_T, X["Credit_Score"]

SMOTE

In [49]:
# Posibilidades: auto, minority, not_minority, not_majority, all
# TODO: Intentar otros metodos de oversampling, intentar undersampling
# TODO: Probar otros parametros
# TODO: EN SERIO??? FUNCIONA MEJOR CON K_NEIGHBORS=1 QUE CON 5???? WTF?? COMPROBAR OVERFITTING

smote = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=1)
#smote = KMeansSMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=1)
#smote = BorderlineSMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=1, m_neighbors=20)
#adasyn = ADASYN(sampling_strategy='not majority', random_state=42, n_neighbors=1)
#smote = SVMSMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=1, m_neighbors=2, n_jobs=-1) # Lol, no termina nunca.

#_X_T, _Credit_Score = smote.fit_resample(X_T, X["Credit_Score"])

In [50]:
#clf = RandomForestClassifier(n_estimators=100, random_state=42)
#clf = clf.fit(_X_T, _Credit_Score)

#model = SelectFromModel(clf, prefit=True)
#_X_T = model.transform(_X_T)

In [63]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks(sampling_strategy='majority', n_jobs=-1)
#_X_T, _Credit_Score = tl.fit_resample(X_T, X["Credit_Score"])

---

<h3><center>Score</center></h3>

---

In [51]:
scoring = {
    "precision_micro" : make_scorer(precision_score, average = 'micro'),
    "recall_micro"    : make_scorer(recall_score,    average = 'micro'),
    "precision_macro" : make_scorer(precision_score, average = 'macro'),
    "recall_macro"    : make_scorer(recall_score,    average = 'macro'),
    "f1_micro"        : make_scorer(f1_score,        average = 'micro'),
    "f1_macro"        : make_scorer(f1_score,        average = 'macro')
}

def print_scores(scores):
    print("f1     :", scores)
    print("f1 avg :", stat.mean(scores))
    print("f1 std :", stat.variance(scores)**(1/2)) # Esto significa algo para F1?

def print_detailed(scores, cv):
    # Añadir medias
    precision_micro = scores["test_precision_micro"].tolist() + [stat.mean(scores["test_precision_micro"]), ""]
    recall_micro    = scores["test_recall_micro"].tolist()    + [stat.mean(scores["test_recall_micro"]),    ""]
    precision_macro = scores["test_precision_macro"].tolist() + [stat.mean(scores["test_precision_macro"]), ""]
    recall_macro    = scores["test_recall_macro"].tolist()    + [stat.mean(scores["test_recall_macro"]),    ""]
    f1_micro        = scores["test_f1_micro"].tolist()        + [stat.mean(scores["test_f1_micro"]),  stat.variance(scores["test_f1_micro"])**(1/2)]
    f1_macro = scores["test_f1_macro"].tolist()               + [stat.mean(scores["test_f1_macro"]),  stat.variance(scores["test_f1_macro"])**(1/2)]
    
    rf_score = pd.DataFrame()
    rf_score.insert(0, "precision_micro", precision_micro)
    rf_score.insert(1, "recall_micro",    recall_micro)
    rf_score.insert(2, "f1_micro",        f1_micro)
    rf_score.insert(3, "precision_macro", precision_macro)
    rf_score.insert(4, "recall_macro",    recall_macro)
    rf_score.insert(5, "f1_macro",        f1_macro)
    
    rf_score.index = [str(n) for n in range(cv)] + ["avg", "std"]
    
    display(rf_score)

---
SGDClassifier

In [54]:
from sklearn.linear_model import SGDClassifier

#Se supone que es un perceptron
clf = SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty=None)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

,precision_micro,recall_micro,f1_micro,precision_macro,recall_macro,f1_macro
0,0.594449,0.594449,0.594449,0.630295,0.594449,0.581895
1,0.612867,0.612867,0.612867,0.639612,0.612867,0.600769
2,0.623319,0.623319,0.623319,0.653001,0.623319,0.606754
3,0.615266,0.615266,0.615266,0.637011,0.615266,0.606298
4,0.618093,0.618093,0.618093,0.659138,0.618093,0.565934
avg,0.612799,0.612799,0.612799,0.643811,0.612799,0.592330
std,,,0.010972,,,0.017889


---
Random Forest

In [79]:
# Stack Overflow dice que f1_mocro es mejor para datasets inbalanceados, aunque lo hemos balanceado
# Sin embargo, al calcular F1 manualmente me sale f1_micro. Asi que quiza sea mejor f1_micro?
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # EXPERIMENTAL: class_weight=balanced

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)


clf.fit(_X_T, _Credit_Score)
pred = clf.predict(Y_T)
solucion = pd.DataFrame(data = { "ID" : Y["ID"], "Credit_Score": pred })
display(solucion)
solucion.to_csv("solucion.csv", index = False)

,precision_micro,recall_micro,f1_micro,precision_macro,recall_macro,f1_macro
0,0.807147,0.807147,0.807147,0.798173,0.80212,0.799800
1,0.81683,0.81683,0.816830,0.805993,0.817237,0.811241
2,0.816421,0.816421,0.816421,0.807395,0.812354,0.809651
3,0.817921,0.817921,0.817921,0.809328,0.812992,0.811121
4,0.811511,0.811511,0.811511,0.803501,0.806329,0.804791
avg,0.813966,0.813966,0.813966,0.804878,0.810207,0.807321
std,,,0.004539,,,0.004955


,ID,Credit_Score
0,99246,Poor
1,90071,Good
2,64375,Poor
3,142441,Poor
4,135010,Standard
...,...,...
19995,126990,Good
19996,128347,Standard
19997,88941,Good
19998,89599,Standard


---
ExtraTrees

In [80]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

,precision_micro,recall_micro,f1_micro,precision_macro,recall_macro,f1_macro
0,0.797736,0.797736,0.797736,0.788238,0.792083,0.789938
1,0.809875,0.809875,0.809875,0.799566,0.807094,0.803166
2,0.807692,0.807692,0.807692,0.798727,0.804505,0.801477
3,0.805101,0.805101,0.805101,0.797007,0.794857,0.795922
4,0.799918,0.799918,0.799918,0.792996,0.792293,0.792582
avg,0.804064,0.804064,0.804064,0.795307,0.798166,0.796617
std,,,0.005129,,,0.005654


---
MLPClassifier

In [81]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=350)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

clf.fit(_X_T, _Credit_Score)
pred = clf.predict(Y_T)
solucion = pd.DataFrame(data = { "ID" : Y["ID"], "Credit_Score": pred })
display(solucion)
solucion.to_csv("solucion.csv", index = False)

,precision_micro,recall_micro,f1_micro,precision_macro,recall_macro,f1_macro
0,0.733361,0.733361,0.733361,0.717886,0.715799,0.716816
1,0.750409,0.750409,0.750409,0.732877,0.738852,0.735147
2,0.734724,0.734724,0.734724,0.723733,0.719421,0.718156
3,0.736361,0.736361,0.736361,0.727651,0.708438,0.716706
4,0.742226,0.742226,0.742226,0.728558,0.719078,0.723148
avg,0.739416,0.739416,0.739416,0.726141,0.720318,0.721994
std,,,0.007013,,,0.007807


,ID,Credit_Score
0,99246,Good
1,90071,Standard
2,64375,Standard
3,142441,Poor
4,135010,Standard
...,...,...
19995,126990,Good
19996,128347,Standard
19997,88941,Standard
19998,89599,Standard


---
Bagging (RF)

In [ ]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(
    RandomForestClassifier(n_estimators=100, random_state=42)
)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=3, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=3)

---
GBC

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, random_state=42)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

---
XGB

In [ ]:
clf = XGBClassifier(seed=42)

scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

---
AdaBoost

In [ ]:
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_validate(clf, _X_T, _Credit_Score, cv=5, n_jobs=-1, scoring=scoring)
print_detailed(scores, cv=5)

---

<h3><center>Entrenamiento</center></h3>

---

In [69]:


clf.fit(_X_T, _Credit_Score)

RandomForestClassifier(random_state=42)

---

<h3><center>Predicción</center></h3>

---

In [70]:
pred = clf.predict(Y_T)


---

<h3><center>CSV</center></h3>

---


In [71]:
solucion = pd.DataFrame(data = { "ID" : Y["ID"], "Credit_Score": pred })
display(solucion)

,ID,Credit_Score
0,99246,Poor
1,90071,Good
2,64375,Poor
3,142441,Poor
4,135010,Good
...,...,...
19995,126990,Good
19996,128347,Standard
19997,88941,Good
19998,89599,Standard


In [73]:
solucion.to_csv("solucion.csv", index = False)

---

<h3><center>Ajuste de hiperparametros </center></h3>

---

In [ ]:
%%script false --no-raise-error

##########################
# HYPEROPT: EXPERIMENTAL #
##########################
search_space={
    "max_depth"    : hp.quniform("max_depth", 5, 20, 1),
    "max_features" : hp.quniform("max_features", 5, 20, 1),
    "n_estimators" : hp.quniform("n_estimators", 100, 200, 1),
    "random_state" : hp.quniform("random_state", 1, 10000000, 1)
}

# define the function we want to minimize
def objective(params):
    clf = RandomForestClassifier(
        #n_jobs=-1,
        n_estimators=int(params["n_estimators"]),
        max_features=int(params["max_features"]),
        max_depth=int(params["max_depth"]),
        random_state=int(params["random_state"]),
        class_weight="balanced" # Experimental
    )

    scores = cross_val_score(clf, _X_T, _Credit_Score, cv=3, n_jobs=3, scoring="f1_macro")

    return {'loss': -stat.mean(scores), 'status': STATUS_OK}

trials = Trials()
best_params = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=200,
  trials=trials
)

print(best_params)

In [ ]:
%%script false --no-raise-error

##################
# RandomsearchCV #
##################
from sklearn.model_selection import RandomizedSearchCV

param_grid={
    "random_state": [randint(0, 1000000000) for x in range(4)] + [42]
}

_clf = RandomForestClassifier(n_estimators=100, n_jobs=6)  # EXPERIMENTAL: class_weight=balanced

cv = RandomizedSearchCV(_clf, param_grid, random_state=42, n_jobs=6, scoring="f1_macro")
cv.fit(_X_T, _Credit_Score)

In [ ]:
%%script false --no-raise-error

################
# GridsearchCV #
################
_clf = RandomForestClassifier(n_estimators=100, n_jobs=3)  # EXPERIMENTAL: class_weight=balanced

cv = GridSearchCV(_clf, scoring="f1_macro", cv=3, n_jobs=3, param_grid={
    "random_state": [randint(0, 1000000000) for x in range(10)] + [42]
})

cv.fit(_X_T, _Credit_Score)

print(cv.cv_results_)
print(cv.best_params_)

---

<h3><center>Analisis de features</center></h3>

---

In [ ]:
%%script false --no-raise-error

##################################################
# Experimental: Informar de features redundantes #
##################################################

const = DropConstantFeatures(tol=0.9)
dupl  = DropDuplicateFeatures()
corr  = SmartCorrelatedSelection(
    threshold=0.8,
    selection_method="model_performance",
    estimator=RandomForestClassifier(n_estimators=100, random_state=42),
    scoring="f1_macro",
    cv=3
)

const.fit(X)
dupl.fit(X)
corr.fit(X, X["Credit_Score"])

print(len(const.features_to_drop_), len(dupl.duplicated_feature_sets_), len(corr.correlated_feature_sets_))
print(const.features_to_drop_)
print(dupl.duplicated_feature_sets_)
print(corr.correlated_feature_sets_)